<style>
*
{
	text-align: justify;
	line-height: 1.5;
	font-family: "Arial", sans-serif;
	font-size: 12px;
}

h2, h3, h4, h5, h6
{
	font-family: "Arial", sans-serif;
	font-size: 12px;
	font-weight: bold;
}
h2
{
	font-size: 14px;
}
h1
{
	font-family: "Wingdings", sans-serif;
	font-size: 16px;
}
</style>

## Milk Production Prediction Model Builder

<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("tahirawwad", "agriculture-data-analytics", "notebooks/notebook-5-03-ml-milk-production.ipynb", "master"))
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/tahirawwad/agriculture-data-analytics/master?filepath=notebooks/notebook-5-03-ml-milk-production.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/tahirawwad/agriculture-data-analytics/blob/master/notebooks/notebook-5-03-ml-milk-production.ipynbynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

### Objective
Create the best machine learning model to predict Milk production value in Ireland according to the historical data from Central Statistics office AEA01 Value at Current Prices for Output, Input and Income in Agriculture.

### Setup

Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
# Local
#!pip install -r script/requirements.txt
# Remote option
#!pip install -r https://raw.githubusercontent.com/tahirawwad/agriculture-data-analytics/requirements.txt
#Options: --quiet --user

In [2]:
from keras_tuner.tuners import RandomSearch
from pandas import read_csv, DataFrame
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from xgboost import XGBRegressor
import numpy as np
import pandas 
import os
import pickle
import shutil

In [3]:
pandas.options.display.float_format = '{:.5f}'.format

In [4]:
READ_BINARY = "rb"
WRITE_BINARY = "wb"

In [5]:
model_directory: str = "milk-production-models"
directory: str = f'./../artifacts/{model_directory}/'

### Load dataframe

In [28]:
dataframe_filepath = "./../artifacts/irish-milk-production-eda-output.csv"
dataframe: DataFrame = read_csv(dataframe_filepath)

### Production of Milk

Extract the milk production features.

In [29]:
feature_columns = [
    'Year', 
    'All Livestock Products - Milk',
    'Taxes on Products',
    'Subsidies on Products', 
    'Compensation of Employees',
    'Contract Work',
    'Entrepreneurial Income',
    'Factor Income',
    'Fixed Capital Consumption - Farm Buildings',
    'Fixed Capital Consumption - Machinery, Equipment, etc',
    'Interest less FISIM', 
    'Operating Surplus', 
    'Livestock - Cattle',
    'Livestock - Sheep', 
    'Land Rental',
    'Intermediate Consumption - Contract Work',
    'Intermediate Consumption - Crop Protection Products',
    'Intermediate Consumption - Energy and Lubricants',
    'Intermediate Consumption - Feeding Stuffs',
    'Intermediate Consumption - Fertilisers',
    'Intermediate Consumption - Financial Intermediation Services Indirect',
    'Intermediate Consumption - Forage Plants',
    'Intermediate Consumption - Maintenance and Repairs',
#    'Intermediate Consumption - Seeds', 'Intermediate Consumption - Services',
    'Intermediate Consumption - Veterinary Expenses',
#   'Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)',
    'Intermediate Consumption - Other Goods and Services'
]

feature_columns1 = [
    'Year',
    'All Livestock Products - Milk',
    'Intermediate Consumption - Energy and Lubricants',
    'Intermediate Consumption - Feeding Stuffs',
    'Intermediate Consumption - Fertilisers',
    'Land Rental',
    'Taxes on Products',
]

dataframe = dataframe[feature_columns]

dataframe.set_index('Year', drop=True, inplace=True)

print(f"Milk production dataset dimensions: {dataframe.shape[1]} Columns, {dataframe.shape[0]} Rows")

dataframe.head()

Milk production dataset dimensions: 24 Columns, 31 Rows


,All Livestock Products - Milk,Taxes on Products,Subsidies on Products,Compensation of Employees,Contract Work,Entrepreneurial Income,Factor Income,Fixed Capital Consumption - Farm Buildings,"Fixed Capital Consumption - Machinery, Equipment, etc",Interest less FISIM,...,Intermediate Consumption - Contract Work,Intermediate Consumption - Crop Protection Products,Intermediate Consumption - Energy and Lubricants,Intermediate Consumption - Feeding Stuffs,Intermediate Consumption - Fertilisers,Intermediate Consumption - Financial Intermediation Services Indirect,Intermediate Consumption - Forage Plants,Intermediate Consumption - Maintenance and Repairs,Intermediate Consumption - Veterinary Expenses,Intermediate Consumption - Other Goods and Services
Year,,,,,,,,,,,,,,,,,,,,,
1990,1316.30000,75.00000,408.90000,377.60000,180.70000,1577.20000,2321.10000,106.40000,325.50000,267.40000,...,180.70000,47.40000,165.00000,642.20000,326.00000,34.00000,553.10000,203.90000,85.20000,182.30000
1991,1258.90000,78.20000,357.30000,362.50000,172.00000,1440.20000,2136.50000,113.20000,331.70000,237.20000,...,172.00000,50.40000,164.80000,646.50000,319.10000,39.00000,525.60000,184.60000,91.90000,185.50000
1992,1373.10000,79.50000,446.00000,336.80000,179.80000,1842.60000,2516.20000,117.80000,332.40000,237.60000,...,179.80000,55.20000,155.40000,668.50000,305.10000,43.00000,533.80000,175.10000,93.70000,183.00000
1993,1439.00000,68.00000,466.40000,339.00000,199.50000,1985.60000,2586.90000,121.90000,335.50000,162.50000,...,199.50000,57.00000,158.90000,733.50000,301.20000,49.00000,572.90000,195.70000,100.20000,193.10000
1994,1446.20000,53.70000,666.00000,345.20000,205.30000,2051.70000,2623.70000,125.00000,346.50000,119.30000,...,205.30000,57.50000,158.80000,818.60000,316.30000,56.00000,604.60000,220.30000,108.70000,200.30000


### Prepare Model Data

#### Select the feature set and the target

Select the feature set and the target 

In [9]:
feature_values = dataframe.iloc[:, 1:].values
target_values = dataframe.iloc[:, 0].values.reshape(-1, 1)

print(f'Features dimension: {np.shape(feature_values)[1]} Columns, {np.shape(feature_values)[0]} Rows')
print(f'Target dimension:   {np.shape(target_values)[1]} Column,  {np.shape(target_values)[0]} Rows')

Features dimension: 23 Columns, 31 Rows
Target dimension:   1 Column,  31 Rows


#### Define Training and Test Sets

The data is liner data and should not be shuffled. Set test set size to 20%.

In [10]:
test_size: float = 0.8
X_train, X_test, Y_train, Y_test = train_test_split(feature_values,
                                                    target_values,
                                                    test_size=test_size,
                                                    shuffle=False)

#### Scale & Transform

In [11]:
features_scaler = MinMaxScaler()

features_scaler.fit(X_train)

xtest_scale = features_scaler.transform(X_test)
xtrain_scale = features_scaler.transform(X_train)

In [12]:
target_scaler = MinMaxScaler()

target_scaler.fit(Y_train)

ytrain_scale = target_scaler.transform(Y_train)
ytest_scale = target_scaler.transform(Y_test)

##### Save Scalers

In [13]:
filename: str = 'milk-features-scaler.pickle'
features_scaler_filepath: str = f'{directory}{filename}'

with open(features_scaler_filepath, WRITE_BINARY) as file:
    pickle.dump(features_scaler, file)

In [14]:
filename: str = 'milk-target-scaler.pickle'
target_scaler_filepath: str = f'{directory}{filename}'

with open(target_scaler_filepath, WRITE_BINARY) as file:
    pickle.dump(target_scaler, file)

### Model Testing

#### Score Board

In [15]:
model_scores_dataframe = DataFrame(
    columns=['Model', 'Mean Absolute Error Score(%)'])

#### Random Forest Regressor

##### Train Model

Hyper parameter tuning via Grid Search Cross Validation

In [16]:
random_forest_regressor = RandomForestRegressor()

random_forest_regressor_paramaters_grid = {
    'bootstrap': [True, False],
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_depth': [1, 3, 5],
    'max_features': ["auto", "sqrt", "log2"],
    'n_estimators': [100, 500, 800],  # Number of trees
}

grid_search_cv = GridSearchCV(
    estimator=random_forest_regressor,
    param_grid=random_forest_regressor_paramaters_grid,
    n_jobs=-1,  # Use all processors on CPU
    cv=5)  #cross validation 5 fold of datasets

grid_search_cv.fit(xtrain_scale, ytrain_scale.reshape(-1))

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['squared_error', 'absolute_error',
                                       'poisson'],
                         'max_depth': [1, 3, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 500, 800]})

##### Test and Score Model

In [17]:
print('Best training model', grid_search_cv.best_estimator_)
print('Best training model score, coefficient of determination R squared',
      grid_search_cv.best_score_)

y_predict = target_scaler.inverse_transform(
    grid_search_cv.predict(xtest_scale).reshape(-1, 1))
mae_score = mean_absolute_error(Y_test, y_predict)

Best training model RandomForestRegressor(max_depth=1)
Best training model score, coefficient of determination R squared nan


##### Save Model Score

In [18]:
values = ['Random Forest Regressor', mae_score]
model_scores_dataframe.loc[len(model_scores_dataframe)] = values
model_scores_dataframe.head()

,Model,Mean Absolute Error Score(%)
0,Random Forest Regressor,347.43159


#### XGBOOST Regressor

##### Train Model

In [19]:
# define XGBRegressor
xgb_regressor_milk = XGBRegressor(random_state=2021)

# define parameters space to loop over
params_xgb_milk = {
    'n_estimators': [20, 40, 80, 160, 340, 500],
    'max_depth': [3, 6, 9],
    'gamma': [0.01, 0.1],
    'learning_rate': [0.001, 0.01, 0.1, 1]
}

# Hyper parameter tuning via Grid Search Cross Validation
grid_xgb_milk = GridSearchCV(
    estimator=xgb_regressor_milk,
    param_grid=params_xgb_milk,
    #n_jobs=-1,
    scoring=['r2', 'neg_root_mean_squared_error'],
    refit='r2',
    n_jobs=-1,
    cv=5,
    verbose=4)

# fit grid to training scaled set
grid_xgb_milk.fit(xtrain_scale, ytrain_scale)

# print best training model & R squared score
print('Best training model ', grid_xgb_milk.best_estimator_)
print('Best model Parameters', grid_xgb_milk.best_params_)
print('Best training model score, coefficient of determination R squared',
      grid_xgb_milk.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best training model  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0.01, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.001, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=20, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=2021,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
Best model Parameters {'gamma': 0.01, 'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 20}
Best training model score, coefficient of determination R squared nan


C:\Users\user\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


##### Test and Score Model

In [20]:
y_predict = target_scaler.inverse_transform(
    grid_xgb_milk.predict(xtest_scale).reshape(-1, 1))

mae_score = mean_absolute_error(Y_test, y_predict)

##### Save Model Score

In [21]:
model_scores_dataframe.loc[len(model_scores_dataframe)] = ['XGBOOST', mae_score]
model_scores_dataframe.head()

,Model,Mean Absolute Error Score(%)
0,Random Forest Regressor,347.43159
1,XGBOOST,373.67662


#### ANN Artificial Neural Network

##### Train Model

In [22]:
#Training & Keras Parameter Tuning

temp_directory: str = './../temp/ANN-tuner/'


# Define ANN model with Hyper parameter variable
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 23)):
        model.add(
            layers.Dense(units=hp.Int('units_' + str(i),
                                      min_value=23,
                                      max_value=600,
                                      step=32),
                         activation='relu'))
        model.add(layers.Dense(1, activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                      loss='mean_absolute_error',
                      metrics=['mean_absolute_error'])
        return model


if os.path.isdir(temp_directory):
    try:
        shutil.rmtree(temp_directory)
    except OSError as exception:
        print(f"Error: {exception.filename} - {exception.strerror}.")

# create a directory to store each iteration of modelling
tuner = RandomSearch(build_model,
                     objective='val_mean_absolute_error',
                     max_trials=5,
                     executions_per_trial=3,
                     directory=temp_directory,
                     project_name='Milk production')

# Defined parameter space to search in
tuner.search_space_summary()

# train trial models and compare with validation set
tuner.search(xtrain_scale,
             ytrain_scale,
             epochs=50,
             validation_data=(xtest_scale, ytest_scale))

# print best 10 models according to val_mean_absolute_error
print('\n')
tuner.results_summary()

# get best model from training trials
bestANNModel = tuner.get_best_models(num_models=1)[0]

# fit best model to training scaled data and scaled test data
bestANNModel.fit(xtrain_scale,
                 ytrain_scale,
                 epochs=50,
                 validation_data=(xtest_scale, ytest_scale))
#Clean up
if os.path.isdir(temp_directory):
    try:
        shutil.rmtree(temp_directory)
    except OSError as exception:
        print(f"Error: {exception.filename} - {exception.strerror}.")

Trial 5 Complete [00h 00m 11s]
val_mean_absolute_error: 0.9210455814997355

Best val_mean_absolute_error So Far: 0.7721548080444336
Total elapsed time: 00h 01m 00s
INFO:tensorflow:Oracle triggered exit


Results summary
Results in ./../temp/ANN-tuner/Milk production
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 7
units_0: 183
learning_rate: 0.001
Score: 0.7721548080444336
Trial summary
Hyperparameters:
num_layers: 22
units_0: 311
learning_rate: 0.0001
Score: 0.9210455814997355
Trial summary
Hyperparameters:
num_layers: 17
units_0: 407
learning_rate: 0.001
Score: 1.0496629277865093
Trial summary
Hyperparameters:
num_layers: 14
units_0: 119
learning_rate: 0.0001
Score: 1.1782824993133545
Trial summary
Hyperparameters:
num_layers: 20
units_0: 215
learning_rate: 0.0001
Score: 1.730723222096761
Epoch 1/50
1/1 [==============================] - 1s 636ms/step - loss: 0.0969 - mean_absolute_error: 0.0969 - val_loss:

1/1 [==============================] - 0s 38ms/step - loss: 0.0102 - mean_absolute_error: 0.0102 - val_loss: 1.2373 - val_mean_absolute_error: 1.2373
Epoch 47/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0118 - mean_absolute_error: 0.0118 - val_loss: 1.2543 - val_mean_absolute_error: 1.2543
Epoch 48/50
1/1 [==============================] - 0s 35ms/step - loss: 0.0134 - mean_absolute_error: 0.0134 - val_loss: 1.2174 - val_mean_absolute_error: 1.2174
Epoch 49/50
1/1 [==============================] - 0s 30ms/step - loss: 0.0072 - mean_absolute_error: 0.0072 - val_loss: 1.1341 - val_mean_absolute_error: 1.1341
Epoch 50/50
1/1 [==============================] - 0s 32ms/step - loss: 0.0098 - mean_absolute_error: 0.0098 - val_loss: 1.1251 - val_mean_absolute_error: 1.1251


##### Test and Score Model

In [23]:
# Predict Milk Production and un-scale back to original values

y_predict = target_scaler.inverse_transform(
    bestANNModel.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n', y_predict)
print('actual milk production values \n', Y_test)

# Calculate Mean Absolute Error
mae_score = mean_absolute_error(Y_test, y_predict)
#print(MAE_xgb)

predicted milk production values 
 [[1460.6936]
 [1416.2872]
 [1423.9714]
 [1419.3169]
 [1479.3212]
 [1338.6945]
 [1366.9196]
 [1389.8002]
 [1458.1382]
 [1321.8748]
 [1261.327 ]
 [1322.0511]
 [1301.0972]
 [1177.6525]
 [1229.2344]
 [1150.2764]
 [1130.2648]
 [1196.225 ]
 [1098.541 ]
 [1529.9607]
 [1807.1439]
 [1950.9237]
 [1839.7607]
 [1769.7694]
 [2140.3806]]
actual milk production values 
 [[1536. ]
 [1414.5]
 [1431.2]
 [1408.8]
 [1447.1]
 [1566.1]
 [1413. ]
 [1431.3]
 [1417.8]
 [1342.1]
 [1332.5]
 [1667.5]
 [1628.1]
 [1106.5]
 [1541.9]
 [1834.8]
 [1629.8]
 [2073.4]
 [2093.1]
 [1881.1]
 [1790.8]
 [2594.1]
 [2556.7]
 [2608.6]
 [2752.7]]


##### Save Model Score

In [24]:
model_scores_dataframe.loc[len(model_scores_dataframe)] = ['ANN', mae_score]
model_scores_dataframe.head()

,Model,Mean Absolute Error Score(%)
0,Random Forest Regressor,347.43159
1,XGBOOST,373.67662
2,ANN,314.58375


### Save Artifacts

Save trained model into binary pickle file to use the model later with new input data from web app

#### Save Models

In [25]:
filename: str = 'milk-ann-model.h5'
ann_filepath: str = f'{directory}{filename}'
bestANNModel.save(ann_filepath, save_format='h5')

```py
# fit and transform in one line
scaler_x.fit_transform(X_train)
# remeber to inverse the scaling on model output
scaler_x.inverse_transform(xtest_scale)

# Read pickle file into variable to use scaler
scaler_x_pkl_ann = pickle.load(open(f'{directory}feature-scaler.pickle', 'rb'))

with open(target_scaler_filepath, READ_BINARY) as file:
    pickle.dump(scaler_y, file)
# Read pickle file into variable to use scaler

with open(target_scaler_filepath, READ_BINARY) as file:
    target_scaler = pickle.load(file)

print(type(target_scaler))

with open(ann_filepath, WRITE_BINARY) as file:
    pickle.dump(bestANNModel, file)
    
pickle.dump(bestANNModel, open(f'{directory}pkl_ann_milk', 'wb'))

# Read pickle file into variable to use model
model_pkl_ann = pickle.load(open(f'{directory}pkl_ann_milk', 'rb'))

## Example using pickle file with saved ANN model

# take input from source as array
data_input_from_webapp = np.array([357.3, 362.5, 172., 1440.2, 2136.5])

# scale input with same scaler as used in model
scale_data_from_webapp = scaler_x.transform(
    data_input_from_webapp.reshape(1, -1))

# predict scaled value
scaled_prediction = bestANNModel.predict(scale_data_from_webapp)

# descale prediction back to normal value
prediction = scaler_y.inverse_transform(scaled_prediction)
print('\n Expected Milk Production is ', prediction[0][0])
```

#### Fill missing values

```py
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
feature_values = imp_mean.fit_transform(feature_values)
target_values = imp_mean.fit_transform(target_values)
```